<a href="https://colab.research.google.com/github/tawaqalt/arbritrary/blob/master/Tawakalitu_Yusuf_Keras_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Problem 1] Sharing and executing the official tutorial model

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.24265102,  0.07631977,  0.6078176 ,  0.44360787,  0.01114468,
        -0.13607776,  0.16177276, -0.36387512, -0.00415626, -0.29822028]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.07297325, 0.10039011, 0.17081167, 0.14494464, 0.09405585,
        0.08117978, 0.10934596, 0.0646423 , 0.09262765, 0.06902879]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.511089

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 2.3218 - accuracy: 0.1121 - 779ms/epoch - 2ms/step


[2.3218331336975098, 0.11209999769926071]

In [10]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [11]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.11813828, 0.07464056, 0.2127665 , 0.07140718, 0.08145025,
        0.08858814, 0.11122575, 0.07887568, 0.07673828, 0.08616927],
       [0.17598452, 0.07657045, 0.12640083, 0.0771206 , 0.12244269,
        0.08520081, 0.14741817, 0.07357255, 0.07413699, 0.04115234],
       [0.08159023, 0.09088317, 0.18980698, 0.09151282, 0.08231628,
        0.09612679, 0.09407604, 0.08749816, 0.09946989, 0.08671959],
       [0.10655832, 0.0496406 , 0.1487284 , 0.09938994, 0.11070856,
        0.05091699, 0.1369828 , 0.15430926, 0.10123526, 0.04152979],
       [0.12079419, 0.07077374, 0.2031578 , 0.09014641, 0.12876342,
        0.08910011, 0.10749895, 0.06340913, 0.06857519, 0.05778109]],
      dtype=float32)>

# [Problem 3] Learning Iris (binary classification) with Keras

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# Loading the Iris dataset
iris = pd.read_csv('Iris.csv')

# Filter for Iris-versicolor and Iris-virginica
iris_binary = iris[(iris['Species'] == 'Iris-versicolor') | (iris['Species'] == 'Iris-virginica')]

# Extract features and labels
X = iris_binary.drop('Species', axis=1)
y = iris_binary['Species']

# Encode labels (0 for Iris-versicolor, 1 for Iris-virginica)
y = np.where(y == 'Iris-versicolor', 0, 1)

# Split into training and testing sets
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_binary = scaler.fit_transform(X_train_binary)
X_test_binary = scaler.transform(X_test_binary)

# Defining the model
model_binary = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(X_train_binary.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compiling the model
loss_fn = tf.keras.losses.BinaryCrossentropy()
model_binary.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Train the model
model_binary.fit(X_train_binary, y_train_binary, epochs=5, batch_size=5, validation_split=0.2)

# Evaluation of the model
loss, accuracy = model_binary.evaluate(X_test_binary, y_test_binary)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Making predictions
predictions_binary = model_binary.predict(X_test_binary)
predicted_classes_binary = (predictions_binary > 0.5).astype(int)
print(predicted_classes_binary.flatten())


Epoch 1/5
13/13 [==============================] - 3s 81ms/step - loss: 0.6641 - accuracy: 0.6250 - val_loss: 0.6360 - val_accuracy: 0.8125
Epoch 2/5
13/13 [==============================] - 0s 11ms/step - loss: 0.6145 - accuracy: 0.8438 - val_loss: 0.6064 - val_accuracy: 0.9375
Epoch 3/5
13/13 [==============================] - 0s 14ms/step - loss: 0.5751 - accuracy: 0.8594 - val_loss: 0.5785 - val_accuracy: 0.8750
Epoch 4/5
13/13 [==============================] - 0s 7ms/step - loss: 0.5395 - accuracy: 0.8906 - val_loss: 0.5517 - val_accuracy: 0.8750
Epoch 5/5
1/1 [==============================] - 0s 54ms/step - loss: 0.4865 - accuracy: 0.9500
Test Loss: 0.4865, Test Accuracy: 0.9500
1/1 [==============================] - 0s 181ms/step
[1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0 0]


# [Problem 4] Learning Iris (multi-level classification) with Keras

In [13]:
# Load the Iris dataset
iris = pd.read_csv('Iris.csv')

# Extracting features and labels
X = iris.drop('Species', axis=1)
y = iris['Species']

# One-hot encoding the labels
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1)).toarray()

# Split into training and testing sets
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_multi = scaler.fit_transform(X_train_multi)
X_test_multi = scaler.transform(X_test_multi)

# Defining the model
model_multi = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(X_train_multi.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compiling the model
loss_fn = tf.keras.losses.CategoricalCrossentropy()
model_multi.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Training the model
model_multi.fit(X_train_multi, y_train_multi, epochs=10, batch_size=5, validation_split=0.2)

# Evaluating the model
loss, accuracy = model_multi.evaluate(X_test_multi, y_test_multi)
print(f'\nTest Loss: {loss:.4f}, \nTest Accuracy: {accuracy:.4f}')

# Make predictions
predictions_multi = model_multi.predict(X_test_multi)
predicted_classes_multi = np.argmax(predictions_multi, axis=1)
print(predicted_classes_multi)


Epoch 1/10
20/20 [==============================] - 2s 25ms/step - loss: 1.0422 - accuracy: 0.4375 - val_loss: 1.0125 - val_accuracy: 0.6250
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.9849 - accuracy: 0.5312 - val_loss: 0.9724 - val_accuracy: 0.5833
Epoch 3/10
20/20 [==============================] - 0s 9ms/step - loss: 0.9390 - accuracy: 0.5729 - val_loss: 0.9302 - val_accuracy: 0.6250
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.8928 - accuracy: 0.6042 - val_loss: 0.8987 - val_accuracy: 0.6667
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.8469 - accuracy: 0.6354 - val_loss: 0.8579 - val_accuracy: 0.7500
Epoch 6/10
20/20 [==============================] - 0s 13ms/step - loss: 0.7975 - accuracy: 0.6875 - val_loss: 0.8155 - val_accuracy: 0.7500
Epoch 7/10
20/20 [==============================] - 0s 13ms/step - loss: 0.7481 - accuracy: 0.6771 - val_loss: 0.7637 - val_accuracy: 0.7500
Epoch 8/10
20/20 

# [Problem 5] Learning House Prices with Keras

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Loading the House Prices dataset
house_prices = pd.read_csv('train.csv')

# Extracting features and labels
X = house_prices[['GrLivArea', 'YearBuilt']]
y = house_prices['SalePrice']

# Splitting into training and testing sets
X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_hp = scaler.fit_transform(X_train_hp)
X_test_hp = scaler.transform(X_test_hp)

# Defining the model
model_hp = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_hp.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compiling the model
loss_fn = tf.keras.losses.MeanSquaredError()
model_hp.compile(optimizer='adam', loss=loss_fn, metrics=['MeanAbsoluteError'])

# Training the model
model_hp.fit(X_train_hp, y_train_hp, epochs=5, batch_size=32, validation_split=0.2)

# Evaluating the model
loss, mae = model_hp.evaluate(X_test_hp, y_test_hp)
print(f'\nTest Mean Absolute Error: ${mae:.2f}')

# Making predictions using the model
predictions_hp = model_hp.predict(X_test_hp)
print(predictions_hp.flatten())


Epoch 1/5
30/30 [==============================] - 3s 16ms/step - loss: 39146835968.0000 - mean_absolute_error: 181526.1875 - val_loss: 37841817600.0000 - val_mean_absolute_error: 181101.2656
Epoch 2/5
30/30 [==============================] - 0s 12ms/step - loss: 39146242048.0000 - mean_absolute_error: 181524.6875 - val_loss: 37840961536.0000 - val_mean_absolute_error: 181099.1406
Epoch 3/5
30/30 [==============================] - 0s 8ms/step - loss: 39144935424.0000 - mean_absolute_error: 181521.3906 - val_loss: 37839130624.0000 - val_mean_absolute_error: 181094.5469
Epoch 4/5
30/30 [==============================] - 0s 13ms/step - loss: 39142330368.0000 - mean_absolute_error: 181514.9062 - val_loss: 37835784192.0000 - val_mean_absolute_error: 181086.1719
Epoch 5/5
10/10 [==============================] - 0s 8ms/step - loss: 39641477120.0000 - mean_absolute_error: 178809.1250

Test Mean Absolute Error: $178809.12
10/10 [==============================] - 0s 4ms/step
[ 20.389345  60.305

# [Problem 6] Learning MNIST with Keras

In [15]:
from tensorflow.keras.datasets import mnist

# Loading the MNIST dataset
(X_train_mnist, y_train_mnist), (X_test_mnist, y_test_mnist) = mnist.load_data()

# Normalizing the images
X_train_mnist = X_train_mnist / 255.0
X_test_mnist = X_test_mnist / 255.0

# Reshaping the images
X_train_mnist = X_train_mnist.reshape(X_train_mnist.shape[0], 28, 28, 1)
X_test_mnist = X_test_mnist.reshape(X_test_mnist.shape[0], 28, 28, 1)

# Defining the model
model_mnist = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)  # Output layer for 10 classes
])

# Compiling the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_mnist.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Training the model
model_mnist.fit(X_train_mnist, y_train_mnist, epochs=5, batch_size=32, validation_data=(X_test_mnist, y_test_mnist))

# Evaluating the model
loss, accuracy = model_mnist.evaluate(X_test_mnist, y_test_mnist)
print(f'\nTest Loss: {loss:.4f}, \nTest Accuracy: {accuracy:.4f}')

# Make predictions using the model
predictions_mnist = model_mnist.predict(X_test_mnist)
predicted_classes_mnist = np.argmax(predictions_mnist, axis=1)
print(predicted_classes_mnist)


Epoch 1/5
1875/1875 [==============================] - 66s 34ms/step - loss: 0.1498 - accuracy: 0.9534 - val_loss: 0.0390 - val_accuracy: 0.9875
Epoch 2/5
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0453 - accuracy: 0.9858 - val_loss: 0.0331 - val_accuracy: 0.9901
Epoch 3/5
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0341 - accuracy: 0.9890 - val_loss: 0.0316 - val_accuracy: 0.9895
Epoch 4/5
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0254 - accuracy: 0.9917 - val_loss: 0.0287 - val_accuracy: 0.9913
Epoch 5/5
313/313 [==============================] - 3s 8ms/step - loss: 0.0296 - accuracy: 0.9911

Test Loss: 0.0296, 
Test Accuracy: 0.9911
313/313 [==============================] - 4s 12ms/step
[7 2 1 ... 4 5 6]


[Problem 7] Rewriting to PyTorch
## 1. Iris Binary Classification with PyTorch

In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Iris dataset
iris = pd.read_csv('Iris.csv')

# Filter for Iris-versicolor and Iris-virginica
iris_binary = iris[(iris['Species'] == 'Iris-versicolor') | (iris['Species'] == 'Iris-virginica')]

# Extract features and labels
X = iris_binary.drop('Species', axis=1).values.astype(np.float32)
y = np.where(iris_binary['Species'] == 'Iris-versicolor', 0, 1).astype(np.float32)

# Split into training and testing sets
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_binary = scaler.fit_transform(X_train_binary)
X_test_binary = scaler.transform(X_test_binary)

# Convert to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train_binary)
y_train_tensor = torch.from_numpy(y_train_binary).view(-1, 1)
X_test_tensor = torch.from_numpy(X_test_binary)
y_test_tensor = torch.from_numpy(y_test_binary).view(-1, 1)

# Define the model
class IrisBinaryModel(nn.Module):
    def __init__(self, input_dim):
        super(IrisBinaryModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)  # Output layer for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Get the number of features
input_dim = X_train_tensor.shape[1]

# Instantiate the model, define loss function and optimizer
model_binary = IrisBinaryModel(input_dim)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model_binary.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model_binary.train()
    optimizer.zero_grad()
    outputs = model_binary(X_train_tensor)
    loss = loss_fn(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model_binary.eval()
with torch.no_grad():
    test_outputs = model_binary(X_test_tensor)
    predicted_classes_binary = (test_outputs > 0.5).int()
    accuracy_binary = (predicted_classes_binary == y_test_tensor).float().mean()
    print(f'Test Accuracy: {accuracy_binary:.4f}')

Epoch [10/100], Loss: 0.6804
Epoch [20/100], Loss: 0.6609
Epoch [30/100], Loss: 0.6385
Epoch [40/100], Loss: 0.6123
Epoch [50/100], Loss: 0.5824
Epoch [60/100], Loss: 0.5486
Epoch [70/100], Loss: 0.5108
Epoch [80/100], Loss: 0.4698
Epoch [90/100], Loss: 0.4268
Epoch [100/100], Loss: 0.3829
Test Accuracy: 1.0000


Pytorch for multiclassification


In [29]:

# Extract features and labels
X = iris.drop('Species', axis=1).values.astype(np.float32)
y = iris['Species'].values

# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Split into training and testing sets
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_multi = scaler.fit_transform(X_train_multi)
X_test_multi = scaler.transform(X_test_multi)

# Convert to PyTorch tensors
X_train_tensor_multi = torch.from_numpy(X_train_multi)
y_train_tensor_multi = torch.from_numpy(y_train_multi).float()
X_test_tensor_multi = torch.from_numpy(X_test_multi)
y_test_tensor_multi = torch.from_numpy(y_test_multi).float()

# Define the model
class IrisMultiModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(IrisMultiModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Get input and output dimensions
input_dim = X_train_tensor_multi.shape[1]
output_dim = y_train_tensor_multi.shape[1]

# Instantiate the model, define loss function and optimizer
model_multi = IrisMultiModel(input_dim, output_dim)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_multi.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model_multi.train()
    optimizer.zero_grad()
    outputs = model_multi(X_train_tensor_multi)
    loss = loss_fn(outputs, y_train_tensor_multi)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model_multi.eval()
with torch.no_grad():
    test_outputs = model_multi(X_test_tensor_multi)
    predicted_classes_multi = test_outputs.argmax(dim=1)
    accuracy_multi = (predicted_classes_multi == y_test_tensor_multi.argmax(dim=1)).float().mean()
    print(f'Test Accuracy: {accuracy_multi:.4f}')

Epoch [10/100], Loss: 1.0523
Epoch [20/100], Loss: 1.0192
Epoch [30/100], Loss: 0.9825
Epoch [40/100], Loss: 0.9421
Epoch [50/100], Loss: 0.8975
Epoch [60/100], Loss: 0.8482
Epoch [70/100], Loss: 0.7956
Epoch [80/100], Loss: 0.7416
Epoch [90/100], Loss: 0.6880
Epoch [100/100], Loss: 0.6361
Test Accuracy: 0.8333


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Pytorch for House Price datasets

In [30]:
 #Load the House Prices dataset
house_prices = pd.read_csv('train.csv')

# Extract features and labels
X = house_prices[['GrLivArea', 'YearBuilt']].values.astype(np.float32)
y = house_prices['SalePrice'].values.astype(np.float32)

# Split into training and testing sets
X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_hp = scaler.fit_transform(X_train_hp)
X_test_hp = scaler.transform(X_test_hp)

# Convert to PyTorch tensors
X_train_tensor_hp = torch.from_numpy(X_train_hp)
y_train_tensor_hp = torch.from_numpy(y_train_hp)
X_test_tensor_hp = torch.from_numpy(X_test_hp)
y_test_tensor_hp = torch.from_numpy(y_test_hp)

# Define the model
class HousePriceModel(nn.Module):
    def __init__(self):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer for regression

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, define loss function and optimizer
model_hp = HousePriceModel()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model_hp.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model_hp.train()
    optimizer.zero_grad()
    outputs = model_hp(X_train_tensor_hp)
    loss = loss_fn(outputs.squeeze(), y_train_tensor_hp)  # Squeeze to match dimensions
    loss.backward()
    optimizer.step()

# Evaluate the model
model_hp.eval()
with torch.no_grad():
    test_outputs_hp = model_hp(X_test_tensor_hp)
    test_loss = loss_fn(test_outputs_hp.squeeze(), y_test_tensor_hp)
    print(f'Test Mean Squared Error: {test_loss:.2f}')

# Make predictions
predictions_hp = test_outputs_hp.squeeze().numpy()
print(predictions_hp)

Test Mean Squared Error: 39648759808.00
[11.201721  21.7411    14.451228   9.973671  14.625707  15.275073
 11.599944   9.797759  15.304946   9.991077  11.763841  12.70478
 14.729979  13.60302   13.62464   12.197957  14.592324  10.864821
 15.338403  13.200096  10.140252  14.669813  13.611158  12.6871195
 12.271913  11.479661  13.837986  13.687302  12.574628  13.620206
 15.129973  16.999222  23.04221   12.538632  14.513419  10.665639
 10.852935  15.962606  20.437073  13.188267  12.666505  16.16946
 11.888495  17.840816  11.931513  18.958698  12.473018  11.967123
 20.216808   9.620141  12.585076   9.981303   9.780431  23.104984
 14.891927  14.877431  13.0765915 10.607633  10.636191  13.7635145
 18.832825  10.359129  21.402485  14.345168  15.348565  12.776225
 13.136816  19.088764  12.426297  12.327332  10.034683  10.17457
 12.90165   12.186345  21.053951  11.745479  19.494267  19.258686
 10.824586  13.604524  11.547257  14.886009  12.315021  14.509998
  9.745408  10.84262   16.585823  10.